In [1]:
from ragcar import Ragcar

In [2]:
Ragcar.available_models("semantic_search")

'Available models for semantic_search are ([src]: model_name_or_path, [model]: leewaay/kpf-bert-base-klueNLI-klueSTS-MSL512, leewaay/klue-roberta-base-klueNLI-klueSTS-MSL512, leewaay/klue-roberta-large-klueNLI-klueSTS-MSL512, MODELS_SUPPORTED(https://huggingface.co/models?pipeline_tag=sentence-similarity)), ([src]: googledrive, [model]: YOUR_MODEL), ([src]: openai, [model]: text-embedding-3-large, text-embedding-3-small, text-embedding-ada-002, MODELS_SUPPORTED(https://platform.openai.com/docs/models)), ([src]: clova, [model]: YOUR_MODEL(https://www.ncloud.com/product/aiService/clovaStudio)), ([src]: elasticsearch, [model]: YOUR_MODEL)'

In [3]:
Ragcar.available_customizable_src("semantic_search")

"Available customizable src for semantic_search are ['elasticsearch', 'googledrive', 'openai', 'model_name_or_path', 'clova']"

## Sentence-Transformers [util.semantic_search](https://www.sbert.net/examples/applications/semantic-search/README.html#util-semantic-search) 활용

In [4]:
retriever = Ragcar(tool="semantic_search", src="model_name_or_path")

In [5]:
corpus = [
    "프로농구 2023-2024시즌, 10월 21일 인삼공사 vs SK 경기로 개막", 
    "LG 최원태, 이적 후 첫 경기서 완벽투…우승 향한 '화룡점정'(종합)",
    "맨시티 홀란, AT 마드리드 친선경기 선발…그리에즈만과 맞대결",
    "이재성, 프리시즌 경기서 골맛…마인츠, 장크트갈렌에 4-1 승",
    "7월 건설경기실사지수 89.8…2년 7개월 만에 최고", 
    "한강∼아라뱃길 복원 모색…서울·인천·경기 '맞손'",
]

query = "류현진, 올라오지 않은 구속·장타에 고전…5이닝 버틴 건 수확(종합)"

In [6]:
retriever(query, corpus)

[[{'corpus_id': 1, 'score': 0.18477356433868408},
  {'corpus_id': 3, 'score': 0.09282227605581284},
  {'corpus_id': 0, 'score': 0.08626362681388855},
  {'corpus_id': 4, 'score': 0.0812467709183693},
  {'corpus_id': 2, 'score': 0.05464627593755722}]]

### Parameter
* top_k (int, optional): The number of top matching entries to retrieve. Defaults to 5.
* min_score (float, optional): The minimum score threshold for an entry to be considered relevant. Defaults to 0.

In [7]:
retriever(query, corpus, top_k=3)

[[{'corpus_id': 1, 'score': 0.18477356433868408},
  {'corpus_id': 3, 'score': 0.09282227605581284},
  {'corpus_id': 0, 'score': 0.08626362681388855}]]

In [8]:
retriever(query, corpus, min_score=0.1)

[[{'corpus_id': 1, 'score': 0.18477356433868408}]]

## Vector DB(Elasticsearch)를 연계한 Semantic Search

### Parameter
* query (str or list or Tensor): query to be encoded
* corpus (str): sentences, paragraphs, documents or Elasticsearch dense_vector field name to be encoded
* top_k (int, optional): retrieve top k matching entries (default: 5)
* min_score (float, optional): minimum similarity (default: 0)
* doc_id (bool, optional): if True, the input query will be treated as an Elasticsearch document ID for retrieval (default: False)
* knn (bool, optional): if True, use k-nearest neighbors algorithm for retrieval (default: False)
* source_fields (list, optional): selected fields from a search (default: [])
* filter (list, optional): list of filters. Each filter should be a dictionary that specifies the field and the value(s) to filter on (default: [])
* must_not (list, optional): list of conditions. Each condition should be a dictionary that specifies the field and the value(s) that should not be matched in the Elasticsearch documents. (default: [])

### 비동기 VS 동기

In [9]:
questions = [
    "아이들과 함께 갈 수 있는 음식점을 추천해줄 수 있나요?", 
    "비가 올 때도 즐길 수 있는 실내 관광지는 어디인가요?",
    "휠체어 사용자를 위한 편의 시설이 잘 갖춰진 음식점을 추천해주세요",
    "제주도 특산물을 맛볼 수 있는 음식점은?",
    "시각 장애인을 위한 음식점"
]

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
Ragcar.available_model_fields("elasticsearch")

'Available fields for elasticsearch are ([field]: encoder_key.src, [type]: str), ([field]: encoder_key.model, [type]: Union), ([field]: host_n, [type]: str), ([field]: http_auth, [type]: tuple, NoneType), ([field]: scheme, [type]: str, NoneType), ([field]: verify_certs, [type]: bool, NoneType), ([field]: timeout, [type]: int, NoneType), ([field]: max_retries, [type]: int, NoneType), ([field]: retry_on_timeout, [type]: bool, NoneType)'

In [12]:
import time


# Output of Sync
print("\n-------------------------\n")
print("Sync")

retriever = Ragcar(
    tool="semantic_search", 
    src="es", 
    model={
        "host_n": os.getenv('ELASTICSEARCH_HOST'),
        "encoder_key": {
            "src": "googledrive", 
            "model": {
                "model_n": "training_klue-stsbenchmark_continue-training_klue-roberta-large_2022-10-31_09-49-34",
                "model_url": "https://drive.google.com/file/d/1F2P-GA9P9PYuUbK_tYotn68P8MbR0EBW/view?usp=drive_link"
            }
        }
    }, 
    use_async=False
)

start = time.time()
hits = retriever(
    questions, 
    "sbert_vector", 
    index_n=os.getenv('ELASTICSEARCH_INDEX'),
    top_k=3, 
    source_fields=["title", "overview_summ"]
)
end = time.time() - start
for hit in hits:
    print(hit)
print(f"Total time: {end}")


async_retriever = Ragcar(
    tool="semantic_search", 
    src="es", 
    model={
        "host_n": os.getenv('ELASTICSEARCH_HOST'),
        "encoder_key": {
            "src": "googledrive", 
            "model": {
                "model_n": "training_klue-stsbenchmark_continue-training_klue-roberta-large_2022-10-31_09-49-34",
                "model_url": "https://drive.google.com/file/d/1F2P-GA9P9PYuUbK_tYotn68P8MbR0EBW/view?usp=drive_link"
            }
        }
    }, 
    use_async=True
)

# Output of Async
print("\n-------------------------\n")
print("Async")
start = time.time()
hits = await async_retriever(
    questions, 
    "sbert_vector", 
    index_n=os.getenv('ELASTICSEARCH_INDEX'),
    top_k=3, 
    source_fields=["title", "overview_summ", "contenttypeid"]
)
end = time.time() - start
for hit in hits:
    print(hit)
print(f"Total time: {end}")


-------------------------

Sync
[{'_index': 'gildong_1', '_id': '12552', '_score': 1.3690594, '_ignored': ['overview.keyword'], '_source': {'overview_summ': '대구광역시 달성군 다사읍 세천리에 위치한 엄마밥상 세천점은 세련된 외관과 고급스러운 분위기를 가지고 있으며, 어린이를 위한 놀이방과 식사할 수 있는 룸이 있어 가족들에게 인기가 있습니다. 대표 메뉴로는 점심특선 엄마밥상이 있으며, 굴전, 고등어, 게장 등 다양한 메뉴를 즐길 수 있습니다. 북다사IC에서 가깝고, 인근에는 마천산산림욕장과 불은사가 있습니다.', 'title': '엄마밥상 세천점'}}, {'_index': 'gildong_1', '_id': '6397', '_score': 1.3228592, '_ignored': ['overview_summ.keyword', 'overview.keyword'], '_source': {'overview_summ': '바비레드 하남점은 스타필드 하남 지하 1층에 있는 퓨전 이탈리안 레스토랑이다. 복합 쇼핑몰인 스타필드에 있어서 가족과 같이 가기 좋은 식당이다. 이곳은 강남역 맛집으로 유명한 바비레드의 하남지점으로 샹들리에 조명의 깔끔하지만 조금은 화려한 느낌의 식당 내부이다. 대표 메뉴는 매운 갈비 파스타이다. 갈비 파스타의 원조 식당으로 매운맛은 주문 시 조절이 가능하다. 키즈 메뉴도 있고 아이들이 색칠 놀이를 할 수 있는 키트와 스티커를 주어 음식을 기다리는 동안 지루해하지 않아 어린이와 같이 가도 편히 음식을 즐길 수 있다.', 'title': '바비레드 하남점'}}, {'_index': 'gildong_1', '_id': '3922', '_score': 1.3152639, '_ignored': ['overview.keyword'], '_source': {'overview_summ': '돈족골은 지하철 9호선 석촌고분역 2번 출구 바

### 데이터 유사성

In [13]:
retriever(
  "16637", 
  "sbert_vector", 
  index_n=os.getenv('ELASTICSEARCH_INDEX'), 
  top_k=3, 
  doc_id=True,
  source_fields=["title", "overview_summ", "contenttypeid"]
)

[{'_index': 'gildong_1',
  '_id': '20697',
  '_score': 1.6915023,
  '_source': {'overview_summ': '광화문 역 근처에 위치한 제주 토속 음식점이다. 제주도에서 직송한 싱싱한 갈치로 만든다.',
   'title': '한라의집',
   'contenttypeid': '음식점'}},
 {'_index': 'gildong_1',
  '_id': '14253',
  '_score': 1.6875191,
  '_source': {'overview_summ': '제주특별자치도 서귀포시에 있는 한식당이다. 대표메뉴는 갈치조림이다. 갈치구이는 제주도의 대표메뉴이다.',
   'title': '우리봉식당',
   'contenttypeid': '음식점'}},
 {'_index': 'gildong_1',
  '_id': '19375',
  '_score': 1.6569988,
  '_source': {'overview_summ': '토끼와거북이는 갈치조림부터 자연산 활어회, 상다리 부러지는 정식 상차림까지 제주도의 다양한 맛을 즐길 수 있는 향토 음식 전문점이다. 제주국제공항 근처에 위치해 제주 여행의 시작과 끝에 방문하기 좋고 넓은 주차장을 갖추고 있어 편리하다. 얼큰하고 깊은 맛의 갈치조림으로 유명하며 다양한 종류의 신선한 회도 함께 맛볼 수 있다. 아침 식사도 가능한 데다가, 규모가 큰 단독 건물 식당이라 단체로 식사하기 좋다. 메뉴가 다양해 골라 먹는 재미가 있는 곳이다.',
   'title': '토끼와거북이',
   'contenttypeid': '음식점'}}]

In [14]:
retriever(
  "16637", 
  "sbert_vector", 
  index_n=os.getenv('ELASTICSEARCH_INDEX'), 
  top_k=3, 
  doc_id=True,
  source_fields=["title", "overview_summ", "contenttypeid"],
  filter=[{"match": {"contenttypeid": "관광지"}}]
)

[{'_index': 'gildong_1',
  '_id': '21805',
  '_score': 1.6153404,
  '_ignored': ['overview.keyword'],
  '_source': {'overview_summ': '1988년 전후로 형성된 남대문 갈치조림 골목은 저렴한 가격으로 매콤하고 얼큰한 갈치조림을 제공하여 맛거리로 유명해졌다. 주변 식당들이 갈치조림으로 간판을 바꾸면서 발전하였고, 원재료는 국산 갈치를 사용하며 각 식당마다 고유의 맛을 제공한다. 이 골목은 남대문시장의 대표 골목으로 알려져 있으며, 직장인들과 시장 상인들에게 필수 점심 코스로 인기가 있다. 일본 관광객들도 많이 찾아오며, 맛에 반해 한국을 다시 방문할 정도로 인기가 있다.',
   'title': '남대문 갈치조림골목',
   'contenttypeid': '관광지'}},
 {'_index': 'gildong_1',
  '_id': '3736',
  '_score': 1.4437943,
  '_source': {'overview_summ': '도두항은 용두암 해안도로와 이호테우 사이에 있는 방파제로, 관탈도와 추자도행 낚시 배들이 출항하는 곳으로 유명하다. 제주 국제공항에서 차로 15분 거리에 있고, 발판도 비교적 좋아 낚시꾼들이 즐겨 찾는다. 제주 시내에서 가깝고, 신선한 해산물이 있는 유명 맛집, 낚시점들이 있어 관광객과 현지인 모두 편리하게 이용할 수 있다. 또한, 유람선과 요트 등 해양 레저 시설도 있어 많은 관광객이 찾는 곳이다.',
   'title': '도두항',
   'contenttypeid': '관광지'}},
 {'_index': 'gildong_1',
  '_id': '8926',
  '_score': 1.4152282,
  '_ignored': ['overview_summ.keyword', 'overview.keyword'],
  '_source': {'overview_summ': '서귀포항은 우리나라 최남단에 자리하고 있으며, 한라산이 있

## Retrieve & Re-Rank

In [15]:
retriever = Ragcar(
    tool="semantic_search", 
    src="es", 
    model={
        "host_n": os.getenv('ELASTICSEARCH_HOST'),
        "encoder_key": {
            "src": "googledrive", 
            "model": {
                "model_n": "training_klue-stsbenchmark_continue-training_klue-roberta-large_2022-10-31_09-49-34",
                "model_url": "https://drive.google.com/file/d/1F2P-GA9P9PYuUbK_tYotn68P8MbR0EBW/view?usp=drive_link"
            }
        }
    }
)

reranker = Ragcar(tool="sentence_similarity", src="model_name_or_path", model="leewaay/kpf-bert-base-klueSTS-cross")

No sentence-transformers model found with name leewaay/kpf-bert-base-klueSTS-cross. Creating a new one with MEAN pooling.


In [16]:
# for yna
import re


# retrieve the query
def search(question):
    # Output of top-k hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-3 Bi-Encoder Retrieval hits")
    
    hits = retriever(
        question, 
        "sbert_vector", 
        index_n=os.getenv('ELASTICSEARCH_INDEX'),
        top_k=10, 
        source_fields=["title", "overview_summ"]
    )
    
    corpus = ["{}, {}".format(hit['_source']['title'], hit['_source']['overview_summ']) for hit in hits]
    
    for idx, hit in enumerate(hits[0:3]):
        print("\t{:.3f}\t{}".format(hit['_score'], corpus[idx]))
        
    
    # Output of top-k hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    
    hits = reranker(question, corpus, sorted=True)
    
    for idx, hit in enumerate(hits[0:3]):
        print("\t{:.3f}\t{}".format(hit[0], corpus[hit[2]]))

In [17]:
search("강원도에서 아이와 익스트림 스포츠를 즐길 수 있는 여행지 추천해줘")


-------------------------

Top-3 Bi-Encoder Retrieval hits
	1.494	또올래캠프, 또올래오토캠핑장은 경기도 가평군 북면에 위치한 캠핑장으로 특히 아이들이 즐길거리가 많아 주로 가족단위로 방문하는 곳이다. 아이들이 좋아하는 야외 놀이터, 레일기차, 탁구장, 방방이, 분수물놀이장과 시원한 계곡에서 물놀이 등을 즐길 수 있다. 계절에 따라 밤 따기, 송어체험 등 계절행사에 참여할 수 있다.
	1.480	영월키즈캠핑장, 영월의 법흥계곡에 위치한 영월키즈캠핑장는 아이들을 위한 다양한 체험 프로그램과 이벤트를 진행하는 캠핑장이다. 아이들은 물론 어른들도 참여 가능하여 온 가족이 캠핑하기 좋다. 바로 옆 계곡에서는 물놀이는 물론 다이빙, 카약을 즐길 수 있다. 총 90여 개의 사이트가 있고, 일반, 타프 사이트, 카라반 사이트로 크게 나눠져 있다. 겨울에는 장박 예약도 가능하고, 현지 농산물이나 닭갈비 밀키트 등을 예약 주문 가능하다.
	1.480	요기는캠핑장, 강원도 영월에 위치한 요기는캠핑장은 아이가 있는 가족들이 즐기기 좋은 캠핑장이다. 시원하고 맑은 계곡과 수영장, 워터슬라이드, 트램펄린 등 지치지 않는 아이들을 위한 놀이기구들이 준비되어 있다. 가을에는 밤 따기 체험, 겨울에는 눈썰매 등 다양한 활동으로 추억을 남길 수 있다.

-------------------------

Top-3 Cross-Encoder Re-ranker hits
	0.901	이사부사자공원&그림책 나라, 가족형 테마공원으로 동해안의 아름다운 절경을 가장 가까이서 감상할 수 있다. 사계절 썰매장인 터비 썰매는 가족, 친구, 연인들에게 동심의 세계를 안겨줄 것이며, 야간에는 추억의 명화를 감상할 수 있다.
	0.892	마운틴코스터, 강원도 평창 용평리조트에 위치한 마운틴코스터는 SNS에서 인기를 얻고 있는 액티비티로, 탑승자가 속도를 조절할 수 있는 롤러코스터이다. 최고 시속 40km로 가속이 가능하며, 아이와 함께 즐길 수 있다. 티켓은 드래